# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
   "com.snapchat.android_11.1.7.81-2109_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
   "com.snapchat.android_11.2.0.68-2110_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
    "com.instagram.android_161.0.0.37.121-248310212_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_161.0.0.37.121-248310203_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_161.0.0.37.121-248310212_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_161.0.0.37.121-248310203_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.6N3',
    'COMMENT_LONGCLICK_HANDLER': 'X.3Dz',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.37i',
    'DM_HOLDER': 'X.4pi',
    'DM_REPLY': 'X.3nq',
    'DM_VOICEMESSAGE': 'X.4fD',
    'DMVIDEOPLAYER': 'X.5LX',
    'FEEDITEM': 'X.1Vr',
    'FRAGMENT': 'androidx.fragment.app.Fragment',
    'HEADERGENERATOR': 'X.0SM',
    'HTTPREQUESTTASKFACTORY': 'X.0qQ',
    'INAPP_NOTIFICATION': 'X.6Fp',
    'IGTV_VIEWER_FRAGMENT': 'X.7D5',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.11A',
    'JACKSON_CREATOR': 'X.0j0',
    'LIVE_POST': 'X.2A4',
    'MAINFEED': 'X.1Qs',
    'MEDIAITEM': 'X.1Vt',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1oV',
    'NOTIFICATION_HANDLER': 'X.2Sx',
    'PHOTOTIMERCONTROLLER': 'X.2tp',
    'POSTDETAIL': 'X.1do',
    'POSTGESTURE_SIMPLEPOST': 'X.27t',
    'POSTGESTURE_CAROUSELIMAGE': 'X.1xS',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2Fp',
    'POSTVIEW': 'X.1dt',
    'PROFILE': 'X.0lS',
    'PROFILEDETAIL': 'X.2mC',
    'PROFILEPIC_LONGCLICK': 'X.2mL',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2Ps',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0nU',
    'NOTIFICATION_RUNNABLE': 'X.6Fq',
    'SERIALSCHEDULER': 'X.0ir',
    'SESSION': 'X.0Mj',
    'SIMPLE_EXECUTOR': 'X.3zk',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1Yp',
    'VIDEOSETTINGSMANAGER': 'X.1gi',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.6N3', '<init>', 'X.297', 'X.6Nj', 'X.0Mj', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.6Mg', 'X.6Iv'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3Dz', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1G5', 'androidx.fragment.app.FragmentActivity', 'X.0Mj', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mj'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mj'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.37i', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.4pi', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.3nq', 'A0L', 'X.3nq', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.3nq', 'A0I', 'X.3nq'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.5LX', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1Vr', 'A00', 'X.0jI'),
    'FRAGMENT_GETACTIVITY': MethodDec('androidx.fragment.app.Fragment', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0SM', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0qQ', 'A03', 'java.lang.String', 'X.0Mj'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.6Fp', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.7D5', 'A00', 'X.7D5'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.7D5', 'A0C', 'X.7D5'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.11A', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0j0', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0j0', '<init>', 'X.FAC'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.2A4', 'A00', 'X.0k9', 'X.2A5'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'ALD'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1Qs', 'configureActionBar', 'X.1Kk'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1Vt', 'A01', 'X.0jI', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0k9', 'X.1Vt'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1oV', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2Sx', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2tp', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1do', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.27t', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.1xS', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2Fp', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1dt', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.2mC', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0lS', 'A04', 'X.0Mj', 'X.0jw'),
    'PROFILE_ONLONGCLICK': MethodDec('X.2mL', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2Ps', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BNs', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0j', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0Q', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0t', 'X.1qB', 'X.1wH'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0nU', 'A00', 'X.0nS', 'X.1HO'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.6Fq', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0ir', 'A02', 'X.0na'),
    'SESSION_GETTOKEN': MethodDec('X.0Mj', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.3zk', 'onFail', 'X.2GG'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.3zk', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.3zk', '<init>', 'X.0Mj'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1Yp', 'A03', 'X.0k9', 'X.0jv'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1gi', 'A00', 'X.0Mj'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Kj', 'A09'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.297', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3Dz', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1aP', 'A0Z'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.37i', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.4py', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.4q0', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.4fD', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1Vr', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0uV', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.7D5', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1Qs', 'A0P'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1VN', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1wH', 'A0A'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1wH', 'A09'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.1ID', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1s6', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.6Fp', 'A00'), FieldDec('X.6Fq', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.6Fp', 'A01'), FieldDec('X.6Fq', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1uR', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1uU', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.2mH', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.2mH', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.2mH', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.2mH', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.2mJ', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.1IC', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2sv', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1E'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1lU', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1wH', 'A0E')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:
"""
c_decs = tuple()
m_decs = tuple()
f_decs = tuple()
"""

'\nc_decs = tuple()\nm_decs = tuple()\nf_decs = tuple()\n'

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1oV#A01() -> X.1oh#A01()
+ Found single candidate for Method. Considering it a match 
	X.0SM#A00(android.content.Context) -> X.0SW#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0t(X.1qB, X.1wH) -> com.instagram.reels.fragment.ReelViewerFragment#A0t(X.1qN, X.1wT)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.inst

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.91 (LX/7D5; -> LX/6uh;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Mj; -> LX/0Mj;
+ Found single candidate for Class. Considering it a match! 
	LX/0ir; -> LX/0j3;
+ Found single candidate for Class. Considering it a match! 
	LX/0qQ; -> LX/0qc;
+ Found single candidate for Class. Considering it a match! 
	LX/0uV; -> LX/0uh;
+ Found single candidate for Class. Considering it a match! 
	LX/11A; -> LX/11M;
+ Found single candidate for Class. Considering it a match! 
	LX/1IC; -> LX/1IO;
+ Found single candidate for Class. Considering it a match! 
	LX/1ID; -> LX/1IP;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/6N3; -> LX/60F;
+ Found single candidate for Class. Considering it a match! 
	LX/3Dz; -> LX/3EU;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/37i; -> LX/38D;
+ Found single candidate for Class. Considering it a match! 
	LX/4pi; -> LX/4wW;
+ Found single candidate for Class. Considering it a match! 
	LX/5LX; -> LX/5S3;
+ Found single candidate for Class. Considering it a match! 
	LX/0j0; -> LX/0jC;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2tp; -> LX/2uG;
+ Found single candidate for Class. Considering it a match! 
	LX/27t; -> LX/28C;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.1xS#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Fp#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.6N3#<init>(X.297, X.6Nj, X.0Mj, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.6Mg, X.6Iv) -> X.60F#<init>(X.29Q, X.6Ez, X.0Mj, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.5zv, X.5zy)
+ Found single candidate for Method. Considering it a match 
	X.3Dz#onLongPress(android.view.MotionEvent) -> X.3EU#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1G5, androidx.fragment.app.FragmentActivity, X.0

+ Found single candidate for Method. Considering it a match 
	X.1Yp#A03(X.0k9, X.0jv) -> X.1Z1#A03(X.0kL, X.0k7)
Could resolve 0 new Classes, 8 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0jv; -> LX/0k7;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1bG; -> LX/1bS;
+ Found single candidate for Class. Considering it a match! 
	LX/2A5; -> LX/2AO;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0uh; -> LX/0ut;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

74 / 145 Classes were matched
56 / 60 Methods were matched
37 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1oV -> X.1oh
• X.0SM -> X.0SW
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3nq -> X.3oM
• X.0nU -> X.0ng
• X.1Qs -> X.1R4
• X.1gi -> X.1gu
• X.2Ps -> X.2QD
• X.2mC -> X.2md
• X.3zk -> X.3yq
• X.6Fp -> X.6Jb
• X.6Fq -> X.6Jc
• X.0Mj -> X.0Mj
• X.0ir -> X.0j3
• X.0qQ -> X.0qc
• X.0uV -> X.0uh
• X.11A -> X.11M
• X.1IC -> X.1IO
• X.1ID -> X.1IP
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Kj -> X.1Kv
• androidx.fragment.app.

• com.facebook.proxygen.JniHandler#sendRequestWithBodyAndEom(org.apache.http.client.methods.HttpUriRequest, byte[], int, int) -> com.facebook.proxygen.JniHandler#sendRequestWithBodyAndEom(org.apache.http.client.methods.HttpUriRequest, byte[], int, int)
• com.instagram.reels.fragment.ReelViewerFragment#BNs(java.lang.Object) -> com.instagram.reels.fragment.ReelViewerFragment#BO2(java.lang.Object)
• com.instagram.reels.fragment.ReelViewerFragment#A0Q(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0Q(com.instagram.reels.fragment.ReelViewerFragment)
• X.0ir#A02(X.0na) -> X.0j3#A02(X.0nm)
• X.3zk#<init>(X.0Mj) -> X.3yq#<init>(X.0Mj)
• com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent) -> com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent)
• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.60F', '<init>', 'X.29Q', 'X.6Ez', 'X.0Mj', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.5zv', 'X.5zy'),
    MethodDec('X.3EU', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1GH', 'androidx.fragment.app.FragmentActivity', 'X.0Mj', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mj),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mj),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagr

    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.11M'
    'X.0jC'
    'X.2AN'
    'X.1R4'
    'X.1W5'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1oh'
    'X.2TI'
    'X.2uG'
    'X.1e0'
    'X.28C'
    # No Match Found for Class 'X.1xS'
    # No Match Found for Class 'X.2Fp'
    'X.1e5'
    'X.0le'
    'X.2md'
    'X.2mm'
    'com.facebook.proxygen.JniHandler'
    'X.2QD'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.0ng'
    'X.6Jc'
    'X.0j3'
    'X.0Mj'
    'X.3yq'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1Z1'
    'X.1gu'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1Kv', 'A09'),
    FieldDec('X.29Q', 'A00'),
    FieldDec('X.3EU', 'A04'),
    FieldDec('X.1ab', 'A0Z'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    FieldDec('com.instagra

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [20]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [21]:
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mj)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mj)
No matching Method found for X.1xS#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2Fp#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1uR#A06
No matching Field found for X.1uU#A02


In [22]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.60F")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3EU")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.38D")

    override val DM_HOLDER = ClassDec("X.4wW")

    override val DM_REPLY = ClassDec("X.3oM")

    override val DM_VOICEMESSAGE = ClassDec("X.4uL")

    override val DMVIDEOPLAYER = ClassDec("X.5S3")

    override val FEEDITEM = ClassDec("X.1W3")

    override val FRAGMENT = ClassDec("androidx.fragment.app.F

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Kv"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.29Q"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3EU"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1ab"])
    override val COMMENT_TEXT = VariableDec<String>("A0Z")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(